Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe.

To create the dataframe:

    The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
    Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
    More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

    If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.
    Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
    In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

4. Submit a link to your Notebook on your Github repository. (10 marks)

Note: There are different website scraping libraries and packages in Python. One of the most common packages is BeautifulSoup. Here is the package's main documentation page: http://beautiful-soup-4.readthedocs.io/en/latest/

The package is so popular that there is a plethora of tutorials and examples of how to use it. Here is a very good Youtube video on how to use the BeautifulSoup package: https://www.youtube.com/watch?v=ng2o98k983k

Use the BeautifulSoup package to transform the data in the table on the Wikipedia page into the above pandas dataframe

In [13]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import requests

In [14]:
!pip install bs4

  Using cached https://files.pythonhosted.org/packages/21/0a/47fdf541c97fd9b6a610cb5fd518175308a7cc60569962e776ac52420387/beautifulsoup4-4.6.3-py3-none-any.whl
Target directory /home/spark/shared/user-libs/python3/bs4-0.0.1.dist-info already exists. Specify --upgrade to force replacement.
Target directory /home/spark/shared/user-libs/python3/beautifulsoup4-4.6.3.dist-info already exists. Specify --upgrade to force replacement.
Target directory /home/spark/shared/user-libs/python3/bs4 already exists. Specify --upgrade to force replacement.


In [15]:
from bs4 import BeautifulSoup as bs

In [16]:
from urllib.request import (
    urlopen, urlparse, urlunparse, urlretrieve)
from bs4 import BeautifulSoup as bs

In [17]:
import urllib.request

opener = urllib.request.FancyURLopener({})
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
f = opener.open(url)
content = f.read()

/opt/ibm/conda/miniconda3/lib/python3.5/site-packages/ipykernel/__main__.py:3: DeprecationWarning: FancyURLopener style of invoking requests is deprecated. Use newer urlopen functions/methods
  app.launch_new_instance()


In [18]:
# using beautiful soup to parse the HTML/XML codes
soup = bs(content,'xml')

In [19]:
table = soup.find('table')

Postcode      = []
Borough       = []
Neighbourhood = []

for tr_cell in table.find_all('tr'):
    
    counter = 1
    Postcode_var      = -1
    Borough_var       = -1
    Neighbourhood_var = -1
    
    for td_cell in tr_cell.find_all('td'):
        if counter == 1: 
            Postcode_var = td_cell.text
        if counter == 2: 
            Borough_var = td_cell.text
            tag_a_Borough = td_cell.find('a')
        if counter == 3: 
            Neighbourhood_var = str(td_cell.text).strip()
            #tag_a_Neighbourhood = td_cell.find('a')
            
        counter +=1
# Ignore cells with a Borough that is Not assigned or None        
    if (Borough_var == 'Not assigned'): 
        continue
    try:
        if (tag_a_Borough is None):
            continue
    except:
        pass
    if(Postcode_var == -1 or Borough_var == -1 or Neighbourhood_var == -1):
        continue
        
    Postcode.append(Postcode_var)
    Borough.append(Borough_var)
    Neighbourhood.append(Neighbourhood_var)

More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [20]:
unique_p = set(Postcode)
print('Unique Postal codes:', len(unique_p))
Postcode_u      = []
Borough_u       = []
Neighbourhood_u = []

for postcode_unique_element in unique_p:
    p_var = ''; b_var = ''; n_var = ''; 
    for postcode_idx, postcode_element in enumerate(Postcode):
        if postcode_unique_element == postcode_element:
            p_var = postcode_element;
            b_var = Borough[postcode_idx]
            if n_var == '': 
                n_var = Neighbourhood[postcode_idx]
            else:
                n_var = n_var + ', ' + Neighbourhood[postcode_idx]
    Postcode_u.append(p_var)
    Borough_u.append(b_var)
    Neighbourhood_u.append(n_var)

Unique Postal codes: 100


Create dataframe

In [21]:
toronto_d = {'PostalCode':Postcode_u, 'Borough':Borough_u, 'Neighbourhood':Neighbourhood_u}
df = pd.DataFrame.from_dict(toronto_d)
df.head(100)

,Borough,Neighbourhood,PostalCode
0,East Toronto,Business reply mail Processing Centre969 Eastern,M7Y
1,Central Toronto,Roselawn,M5N
2,East York,Leaside,M4G
3,York,Weston,M9N
4,North York,Hillcrest Village,M2H
5,Etobicoke,Westmount,M9P
6,North York,"Northwood Park, York University",M3J
7,West Toronto,"High Park, The Junction South",M6P
8,North York,Downsview Central,M3M
9,Central Toronto,Lawrence Park,M4N


In [22]:
df.sort_values(by=['Borough'])

,Borough,Neighbourhood,PostalCode
1,Central Toronto,Roselawn,M5N
95,Central Toronto,"Forest Hill North, Forest Hill West",M5P
90,Central Toronto,Davisville,M4S
86,Central Toronto,North Toronto West,M4R
98,Central Toronto,"Moore Park, Summerhill East",M4T
47,Central Toronto,"The Annex, North Midtown, Yorkville",M5R
61,Central Toronto,Davisville North,M4P
9,Central Toronto,Lawrence Park,M4N
75,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",M4V
99,Downtown Toronto,"Chinatown, Grange Park, Kensington Market",M5T


If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [23]:
df.loc[df.Neighbourhood == 'Not assigned', 'Neighbourhood'] = df.Borough

In [24]:
df.sort_values(by=['Borough'])

,Borough,Neighbourhood,PostalCode
1,Central Toronto,Roselawn,M5N
95,Central Toronto,"Forest Hill North, Forest Hill West",M5P
90,Central Toronto,Davisville,M4S
86,Central Toronto,North Toronto West,M4R
98,Central Toronto,"Moore Park, Summerhill East",M4T
47,Central Toronto,"The Annex, North Midtown, Yorkville",M5R
61,Central Toronto,Davisville North,M4P
9,Central Toronto,Lawrence Park,M4N
75,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",M4V
99,Downtown Toronto,"Chinatown, Grange Park, Kensington Market",M5T


In [25]:
df.shape

(100, 3)